In [49]:
import nltk
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import gensim

In [50]:
df = pd.read_csv('../datasets/news_dataset.csv')
df.head()

,unique_id,outlet,headline,url,article_text,image,nlp_label,nlp-image_label
0,39e6104d56,CBC.ca,"Tij Iginla embraces family name, eager to foll...",https://www.cbc.ca,Tij Iginla doesn't shy away from his famous la...,images/39e6104d56.jpeg,Likely to be Bias,Likely to be Unbiased
1,89ce50166b,CBC.ca,"The Sunday Magazine for June 23, 2024 - CBC.ca",https://www.cbc.ca,This week on The Sunday Magazine with host Piy...,images/89ce50166b.jpeg,Likely to be Bias,Likely to be Unbiased
2,8b320e107e,CBC News,"Conservative MP shares inaccurate, ChatGPT-gen...",https://www.cbc.ca,An Ontario Conservative MP's use of ChatGPT to...,images/8b320e107e.jpeg,Likely to be Bias,Likely to be Bias
3,7536f87654,CBC.ca,"Women's sports are more popular than ever, but...",https://www.cbc.ca,"When it comes to the study of sports, a man's ...",images/7536f87654.jpeg,Likely to be Unbiased,Likely to be Unbiased
4,c829d1f9a8,CBC.ca,June 22: Listener Question Show - CBC.ca,https://www.cbc.ca,"To wrap up the 48th season of Quirks & Quarks,...",images/c829d1f9a8.jpeg,Likely to be Bias,Likely to be Unbiased


In [51]:
# drop nulls
df = df.dropna()
df.shape

(40945, 8)

In [52]:
# make lowercase
for index, row in df.iterrows():
    df.at[index, "headline_cleaned"] = ''.join([i for i in row["headline"].lower() if not i.isdigit()])
       
df.head()

,unique_id,outlet,headline,url,article_text,image,nlp_label,nlp-image_label,headline_cleaned
0,39e6104d56,CBC.ca,"Tij Iginla embraces family name, eager to foll...",https://www.cbc.ca,Tij Iginla doesn't shy away from his famous la...,images/39e6104d56.jpeg,Likely to be Bias,Likely to be Unbiased,"tij iginla embraces family name, eager to foll..."
1,89ce50166b,CBC.ca,"The Sunday Magazine for June 23, 2024 - CBC.ca",https://www.cbc.ca,This week on The Sunday Magazine with host Piy...,images/89ce50166b.jpeg,Likely to be Bias,Likely to be Unbiased,"the sunday magazine for june , - cbc.ca"
2,8b320e107e,CBC News,"Conservative MP shares inaccurate, ChatGPT-gen...",https://www.cbc.ca,An Ontario Conservative MP's use of ChatGPT to...,images/8b320e107e.jpeg,Likely to be Bias,Likely to be Bias,"conservative mp shares inaccurate, chatgpt-gen..."
3,7536f87654,CBC.ca,"Women's sports are more popular than ever, but...",https://www.cbc.ca,"When it comes to the study of sports, a man's ...",images/7536f87654.jpeg,Likely to be Unbiased,Likely to be Unbiased,"women's sports are more popular than ever, but..."
4,c829d1f9a8,CBC.ca,June 22: Listener Question Show - CBC.ca,https://www.cbc.ca,"To wrap up the 48th season of Quirks & Quarks,...",images/c829d1f9a8.jpeg,Likely to be Bias,Likely to be Unbiased,june : listener question show - cbc.ca


In [53]:
# tokenization
# for title, split into words
df_headline = df.copy()
df_headline["headline_words"] = df_headline["headline_cleaned"].apply(nltk.word_tokenize)
df_headline.head()

,unique_id,outlet,headline,url,article_text,image,nlp_label,nlp-image_label,headline_cleaned,headline_words
0,39e6104d56,CBC.ca,"Tij Iginla embraces family name, eager to foll...",https://www.cbc.ca,Tij Iginla doesn't shy away from his famous la...,images/39e6104d56.jpeg,Likely to be Bias,Likely to be Unbiased,"tij iginla embraces family name, eager to foll...","[tij, iginla, embraces, family, name, ,, eager..."
1,89ce50166b,CBC.ca,"The Sunday Magazine for June 23, 2024 - CBC.ca",https://www.cbc.ca,This week on The Sunday Magazine with host Piy...,images/89ce50166b.jpeg,Likely to be Bias,Likely to be Unbiased,"the sunday magazine for june , - cbc.ca","[the, sunday, magazine, for, june, ,, -, cbc.ca]"
2,8b320e107e,CBC News,"Conservative MP shares inaccurate, ChatGPT-gen...",https://www.cbc.ca,An Ontario Conservative MP's use of ChatGPT to...,images/8b320e107e.jpeg,Likely to be Bias,Likely to be Bias,"conservative mp shares inaccurate, chatgpt-gen...","[conservative, mp, shares, inaccurate, ,, chat..."
3,7536f87654,CBC.ca,"Women's sports are more popular than ever, but...",https://www.cbc.ca,"When it comes to the study of sports, a man's ...",images/7536f87654.jpeg,Likely to be Unbiased,Likely to be Unbiased,"women's sports are more popular than ever, but...","[women, 's, sports, are, more, popular, than, ..."
4,c829d1f9a8,CBC.ca,June 22: Listener Question Show - CBC.ca,https://www.cbc.ca,"To wrap up the 48th season of Quirks & Quarks,...",images/c829d1f9a8.jpeg,Likely to be Bias,Likely to be Unbiased,june : listener question show - cbc.ca,"[june, :, listener, question, show, -, cbc.ca]"


In [54]:
# remove punctuation & numbers
for index, row in df_headline.iterrows():
    df_headline.at[index, "headline_words"] = [w for w in row["headline_words"] if re.search(r'^[a-z]+$', w)]
df_headline.head()

,unique_id,outlet,headline,url,article_text,image,nlp_label,nlp-image_label,headline_cleaned,headline_words
0,39e6104d56,CBC.ca,"Tij Iginla embraces family name, eager to foll...",https://www.cbc.ca,Tij Iginla doesn't shy away from his famous la...,images/39e6104d56.jpeg,Likely to be Bias,Likely to be Unbiased,"tij iginla embraces family name, eager to foll...","[tij, iginla, embraces, family, name, eager, t..."
1,89ce50166b,CBC.ca,"The Sunday Magazine for June 23, 2024 - CBC.ca",https://www.cbc.ca,This week on The Sunday Magazine with host Piy...,images/89ce50166b.jpeg,Likely to be Bias,Likely to be Unbiased,"the sunday magazine for june , - cbc.ca","[the, sunday, magazine, for, june]"
2,8b320e107e,CBC News,"Conservative MP shares inaccurate, ChatGPT-gen...",https://www.cbc.ca,An Ontario Conservative MP's use of ChatGPT to...,images/8b320e107e.jpeg,Likely to be Bias,Likely to be Bias,"conservative mp shares inaccurate, chatgpt-gen...","[conservative, mp, shares, inaccurate, stats, ..."
3,7536f87654,CBC.ca,"Women's sports are more popular than ever, but...",https://www.cbc.ca,"When it comes to the study of sports, a man's ...",images/7536f87654.jpeg,Likely to be Unbiased,Likely to be Unbiased,"women's sports are more popular than ever, but...","[women, sports, are, more, popular, than, ever..."
4,c829d1f9a8,CBC.ca,June 22: Listener Question Show - CBC.ca,https://www.cbc.ca,"To wrap up the 48th season of Quirks & Quarks,...",images/c829d1f9a8.jpeg,Likely to be Bias,Likely to be Unbiased,june : listener question show - cbc.ca,"[june, listener, question, show]"


In [55]:
# remove stopwords
stop_list = set(stopwords.words('english'))
for index, row in df_headline.iterrows():
    df_headline.at[index, "headline_words"] = [w for w in row["headline_words"] if w not in stop_list]

df_headline.head()

,unique_id,outlet,headline,url,article_text,image,nlp_label,nlp-image_label,headline_cleaned,headline_words
0,39e6104d56,CBC.ca,"Tij Iginla embraces family name, eager to foll...",https://www.cbc.ca,Tij Iginla doesn't shy away from his famous la...,images/39e6104d56.jpeg,Likely to be Bias,Likely to be Unbiased,"tij iginla embraces family name, eager to foll...","[tij, iginla, embraces, family, name, eager, f..."
1,89ce50166b,CBC.ca,"The Sunday Magazine for June 23, 2024 - CBC.ca",https://www.cbc.ca,This week on The Sunday Magazine with host Piy...,images/89ce50166b.jpeg,Likely to be Bias,Likely to be Unbiased,"the sunday magazine for june , - cbc.ca","[sunday, magazine, june]"
2,8b320e107e,CBC News,"Conservative MP shares inaccurate, ChatGPT-gen...",https://www.cbc.ca,An Ontario Conservative MP's use of ChatGPT to...,images/8b320e107e.jpeg,Likely to be Bias,Likely to be Bias,"conservative mp shares inaccurate, chatgpt-gen...","[conservative, mp, shares, inaccurate, stats, ..."
3,7536f87654,CBC.ca,"Women's sports are more popular than ever, but...",https://www.cbc.ca,"When it comes to the study of sports, a man's ...",images/7536f87654.jpeg,Likely to be Unbiased,Likely to be Unbiased,"women's sports are more popular than ever, but...","[women, sports, popular, ever, study, female, ..."
4,c829d1f9a8,CBC.ca,June 22: Listener Question Show - CBC.ca,https://www.cbc.ca,"To wrap up the 48th season of Quirks & Quarks,...",images/c829d1f9a8.jpeg,Likely to be Bias,Likely to be Unbiased,june : listener question show - cbc.ca,"[june, listener, question, show]"


In [56]:
# lemmatization
lemmatizer = WordNetLemmatizer()
for index, row in df_headline.iterrows():
    df_headline.at[index, "headline_words"] = [lemmatizer.lemmatize(w) for w in row["headline_words"]]

df_headline.head()

,unique_id,outlet,headline,url,article_text,image,nlp_label,nlp-image_label,headline_cleaned,headline_words
0,39e6104d56,CBC.ca,"Tij Iginla embraces family name, eager to foll...",https://www.cbc.ca,Tij Iginla doesn't shy away from his famous la...,images/39e6104d56.jpeg,Likely to be Bias,Likely to be Unbiased,"tij iginla embraces family name, eager to foll...","[tij, iginla, embrace, family, name, eager, fo..."
1,89ce50166b,CBC.ca,"The Sunday Magazine for June 23, 2024 - CBC.ca",https://www.cbc.ca,This week on The Sunday Magazine with host Piy...,images/89ce50166b.jpeg,Likely to be Bias,Likely to be Unbiased,"the sunday magazine for june , - cbc.ca","[sunday, magazine, june]"
2,8b320e107e,CBC News,"Conservative MP shares inaccurate, ChatGPT-gen...",https://www.cbc.ca,An Ontario Conservative MP's use of ChatGPT to...,images/8b320e107e.jpeg,Likely to be Bias,Likely to be Bias,"conservative mp shares inaccurate, chatgpt-gen...","[conservative, mp, share, inaccurate, stats, c..."
3,7536f87654,CBC.ca,"Women's sports are more popular than ever, but...",https://www.cbc.ca,"When it comes to the study of sports, a man's ...",images/7536f87654.jpeg,Likely to be Unbiased,Likely to be Unbiased,"women's sports are more popular than ever, but...","[woman, sport, popular, ever, study, female, p..."
4,c829d1f9a8,CBC.ca,June 22: Listener Question Show - CBC.ca,https://www.cbc.ca,"To wrap up the 48th season of Quirks & Quarks,...",images/c829d1f9a8.jpeg,Likely to be Bias,Likely to be Unbiased,june : listener question show - cbc.ca,"[june, listener, question, show]"


In [57]:
# can consider implementing n-grams; not implemented now as 
# it will add more dimensionality to the data and can result in vv
# long run times for the models

In [58]:
# add a new col to convert the lsit of words back to a sentence; 
# this is used for BERTopic
df_headline["headline_sentence"] = df_headline["headline_words"].apply(lambda x: ' '.join(x))

df_headline.head()

,unique_id,outlet,headline,url,article_text,image,nlp_label,nlp-image_label,headline_cleaned,headline_words,headline_sentence
0,39e6104d56,CBC.ca,"Tij Iginla embraces family name, eager to foll...",https://www.cbc.ca,Tij Iginla doesn't shy away from his famous la...,images/39e6104d56.jpeg,Likely to be Bias,Likely to be Unbiased,"tij iginla embraces family name, eager to foll...","[tij, iginla, embrace, family, name, eager, fo...",tij iginla embrace family name eager follow ha...
1,89ce50166b,CBC.ca,"The Sunday Magazine for June 23, 2024 - CBC.ca",https://www.cbc.ca,This week on The Sunday Magazine with host Piy...,images/89ce50166b.jpeg,Likely to be Bias,Likely to be Unbiased,"the sunday magazine for june , - cbc.ca","[sunday, magazine, june]",sunday magazine june
2,8b320e107e,CBC News,"Conservative MP shares inaccurate, ChatGPT-gen...",https://www.cbc.ca,An Ontario Conservative MP's use of ChatGPT to...,images/8b320e107e.jpeg,Likely to be Bias,Likely to be Bias,"conservative mp shares inaccurate, chatgpt-gen...","[conservative, mp, share, inaccurate, stats, c...",conservative mp share inaccurate stats capital...
3,7536f87654,CBC.ca,"Women's sports are more popular than ever, but...",https://www.cbc.ca,"When it comes to the study of sports, a man's ...",images/7536f87654.jpeg,Likely to be Unbiased,Likely to be Unbiased,"women's sports are more popular than ever, but...","[woman, sport, popular, ever, study, female, p...",woman sport popular ever study female physiolo...
4,c829d1f9a8,CBC.ca,June 22: Listener Question Show - CBC.ca,https://www.cbc.ca,"To wrap up the 48th season of Quirks & Quarks,...",images/c829d1f9a8.jpeg,Likely to be Bias,Likely to be Unbiased,june : listener question show - cbc.ca,"[june, listener, question, show]",june listener question show


In [59]:
# create new df with the relevant cols
df = df_headline[["outlet", "headline", "nlp_label", "nlp-image_label", 
                  "headline_words", "headline_sentence"]]
df.head()


,outlet,headline,nlp_label,nlp-image_label,headline_words,headline_sentence
0,CBC.ca,"Tij Iginla embraces family name, eager to foll...",Likely to be Bias,Likely to be Unbiased,"[tij, iginla, embrace, family, name, eager, fo...",tij iginla embrace family name eager follow ha...
1,CBC.ca,"The Sunday Magazine for June 23, 2024 - CBC.ca",Likely to be Bias,Likely to be Unbiased,"[sunday, magazine, june]",sunday magazine june
2,CBC News,"Conservative MP shares inaccurate, ChatGPT-gen...",Likely to be Bias,Likely to be Bias,"[conservative, mp, share, inaccurate, stats, c...",conservative mp share inaccurate stats capital...
3,CBC.ca,"Women's sports are more popular than ever, but...",Likely to be Unbiased,Likely to be Unbiased,"[woman, sport, popular, ever, study, female, p...",woman sport popular ever study female physiolo...
4,CBC.ca,June 22: Listener Question Show - CBC.ca,Likely to be Bias,Likely to be Unbiased,"[june, listener, question, show]",june listener question show


In [60]:
df.shape

(40945, 6)

In [61]:
# convert back to csv
df.to_csv('../datasets/news_dataset_headlines_cleaned.csv', index=False)